<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/YouTubeVideoRecommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
from transformers import BertTokenizer, TFBertModel
import torch
import re
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
df_yt=pd.read_csv('/content/US_videos_data.csv')
df_yt.head()
df_yt=df_yt.drop(['Unnamed: 0'],axis=1)

In [30]:
df_yt.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,IhO1FcjDMV4,Meatpacking: Last Week Tonight with John Olive...,2021-02-22T07:30:01Z,UC3XTzVzaHQEd30rQbuvCtTQ,LastWeekTonight,24,21.22.02,[none],1067147,60111,1221,6866,https://i.ytimg.com/vi/IhO1FcjDMV4/default.jpg,False,False,The pandemic has thrown into high relief some ...
1,p4Wy84AOzj0,"Best 3D Pen Art Wins $5,000 Challenge! | ZHC C...",2021-02-21T20:43:28Z,UCPAk4rqVIwg1NCXh61VJxbg,ZHC Crafts,26,21.22.02,[none],1047854,50662,690,4879,https://i.ytimg.com/vi/p4Wy84AOzj0/default.jpg,False,False,I can't believe we made art using 3d pens and ...
2,4eKXwKDdXYA,100 Days - [Minecraft Superflat],2021-02-20T18:00:01Z,UC9FkeEFIGd9FXRfxpTltXYA,Luke TheNotable,20,21.22.02,luke thenotable|luke|the|notable|luke thenotab...,6133266,372753,7961,39551,https://i.ytimg.com/vi/4eKXwKDdXYA/default.jpg,False,False,This video is intended for audiences 13+ years...
3,XHR5mt2gBjo,Amazing! Luke Bryan Calls 15-Year-Old Casey Bi...,2021-02-22T01:53:12Z,UCAMPco9PqjBbI_MLsDOO4Jw,American Idol,24,21.22.02,American Idol|singing competition|Katy Perry|R...,790238,14267,129,1257,https://i.ytimg.com/vi/XHR5mt2gBjo/default.jpg,False,False,Small but MIGHTY! Casey Bishop completely blow...
4,C-icyHEb7W4,Game Theory: Did Reddit Just SOLVE FNAF?,2021-02-20T19:05:26Z,UCo_IB5145EVNcf8hw1Kku7w,The Game Theorists,20,21.22.02,fnaf|five nights at freddy's|fnaf 4|fnaf theor...,3248661,225780,2872,31885,https://i.ytimg.com/vi/C-icyHEb7W4/default.jpg,False,False,Get Yourself Some BRAND NEW Theory Wear! ► htt...


In [31]:
df_yt=df_yt[['title','channelTitle','likes','dislikes','thumbnail_link','description']]
df_yt.head()

,title,channelTitle,likes,dislikes,thumbnail_link,description
0,Meatpacking: Last Week Tonight with John Olive...,LastWeekTonight,60111,1221,https://i.ytimg.com/vi/IhO1FcjDMV4/default.jpg,The pandemic has thrown into high relief some ...
1,"Best 3D Pen Art Wins $5,000 Challenge! | ZHC C...",ZHC Crafts,50662,690,https://i.ytimg.com/vi/p4Wy84AOzj0/default.jpg,I can't believe we made art using 3d pens and ...
2,100 Days - [Minecraft Superflat],Luke TheNotable,372753,7961,https://i.ytimg.com/vi/4eKXwKDdXYA/default.jpg,This video is intended for audiences 13+ years...
3,Amazing! Luke Bryan Calls 15-Year-Old Casey Bi...,American Idol,14267,129,https://i.ytimg.com/vi/XHR5mt2gBjo/default.jpg,Small but MIGHTY! Casey Bishop completely blow...
4,Game Theory: Did Reddit Just SOLVE FNAF?,The Game Theorists,225780,2872,https://i.ytimg.com/vi/C-icyHEb7W4/default.jpg,Get Yourself Some BRAND NEW Theory Wear! ► htt...


In [32]:
df_yt.shape

(6000, 6)

In [33]:
df_yt.isnull().sum()

,0
title,0
channelTitle,0
likes,0
dislikes,0
thumbnail_link,0
description,15


In [34]:
df_yt.duplicated().sum()

18

In [35]:
df_yt=df_yt.drop_duplicates(subset=['title'])

In [36]:
df_yt.duplicated().sum()

0

In [37]:
df_yt.isnull().sum()

,0
title,0
channelTitle,0
likes,0
dislikes,0
thumbnail_link,0
description,4


In [38]:
df_yt=df_yt.dropna()

In [39]:
df_yt.isnull().sum()

,0
title,0
channelTitle,0
likes,0
dislikes,0
thumbnail_link,0
description,0


In [40]:
df_yt['title']

,title
0,Meatpacking: Last Week Tonight with John Olive...
1,"Best 3D Pen Art Wins $5,000 Challenge! | ZHC C..."
2,100 Days - [Minecraft Superflat]
3,Amazing! Luke Bryan Calls 15-Year-Old Casey Bi...
4,Game Theory: Did Reddit Just SOLVE FNAF?
...,...
5840,Zack Snyder’s Justice League | Making the Snyd...
5842,OLYMPIQUE LYONNAIS - PARIS SAINT-GERMAIN (2 - ...
5847,Leicester City v Manchester United | Key Momen...
5851,Freezing Giant water balloon trial 1 #shorts


In [41]:
df_yt['clean_title']=df_yt['title'].apply(lambda x:re.sub(r'[^a-zA-Z0-9\s]','',x) if isinstance(x,str) else '')

In [42]:
df_yt['clean_title']

,clean_title
0,Meatpacking Last Week Tonight with John Oliver...
1,Best 3D Pen Art Wins 5000 Challenge ZHC Crafts
2,100 Days Minecraft Superflat
3,Amazing Luke Bryan Calls 15YearOld Casey Bisho...
4,Game Theory Did Reddit Just SOLVE FNAF
...,...
5840,Zack Snyders Justice League Making the Snyder...
5842,OLYMPIQUE LYONNAIS PARIS SAINTGERMAIN 2 4 H...
5847,Leicester City v Manchester United Key Moment...
5851,Freezing Giant water balloon trial 1 shorts


In [43]:
# Load the hugging face BERT tokenizer and model for TensorFlow
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are i

In [47]:
def get_bert_embeddings(text, tokenizer, model):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=512)

    # Get the model's outputs
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

    # Return the pooled output as numpy array
    return outputs.pooler_output.numpy()

# Apply the function to each clean_title in the dataframe
df_yt['embeddings'] = df_yt['clean_title'].apply(lambda x: get_bert_embeddings(x, tokenizer, model))


In [48]:
df_yt

,title,channelTitle,likes,dislikes,thumbnail_link,description,clean_title,embeddings
0,Meatpacking: Last Week Tonight with John Olive...,LastWeekTonight,60111,1221,https://i.ytimg.com/vi/IhO1FcjDMV4/default.jpg,The pandemic has thrown into high relief some ...,Meatpacking Last Week Tonight with John Oliver...,"[[-0.8381989, -0.49441865, -0.6015343, 0.63540..."
1,"Best 3D Pen Art Wins $5,000 Challenge! | ZHC C...",ZHC Crafts,50662,690,https://i.ytimg.com/vi/p4Wy84AOzj0/default.jpg,I can't believe we made art using 3d pens and ...,Best 3D Pen Art Wins 5000 Challenge ZHC Crafts,"[[-0.8337093, -0.49846038, -0.80845255, 0.6898..."
2,100 Days - [Minecraft Superflat],Luke TheNotable,372753,7961,https://i.ytimg.com/vi/4eKXwKDdXYA/default.jpg,This video is intended for audiences 13+ years...,100 Days Minecraft Superflat,"[[-0.82278955, -0.38171306, -0.6324972, 0.7180..."
3,Amazing! Luke Bryan Calls 15-Year-Old Casey Bi...,American Idol,14267,129,https://i.ytimg.com/vi/XHR5mt2gBjo/default.jpg,Small but MIGHTY! Casey Bishop completely blow...,Amazing Luke Bryan Calls 15YearOld Casey Bisho...,"[[-0.85991377, -0.31500304, 0.18498333, 0.5308..."
4,Game Theory: Did Reddit Just SOLVE FNAF?,The Game Theorists,225780,2872,https://i.ytimg.com/vi/C-icyHEb7W4/default.jpg,Get Yourself Some BRAND NEW Theory Wear! ► htt...,Game Theory Did Reddit Just SOLVE FNAF,"[[-0.8647453, -0.414297, -0.582271, 0.7147872,..."
...,...,...,...,...,...,...,...,...
5840,Zack Snyder’s Justice League | Making the Snyd...,HBO Max,76755,513,https://i.ytimg.com/vi/xu8SB_xte9M/default.jpg,Director Zack Snyder along with actors Henry C...,Zack Snyders Justice League Making the Snyder...,"[[-0.82345307, -0.44228116, -0.37553453, 0.600..."
5842,OLYMPIQUE LYONNAIS - PARIS SAINT-GERMAIN (2 - ...,Ligue 1 Uber Eats Official,11589,333,https://i.ytimg.com/vi/F5nMRXAIONI/default.jpg,OLYMPIQUE LYONNAIS vs PARIS SAINT-GERMAIN High...,OLYMPIQUE LYONNAIS PARIS SAINTGERMAIN 2 4 H...,"[[-0.79725754, -0.38263875, -0.4323063, 0.4329..."
5847,Leicester City v Manchester United | Key Momen...,The Emirates FA Cup,15000,816,https://i.ytimg.com/vi/g4XLge7Pmdo/default.jpg,Leicester City hosted Manchester United in the...,Leicester City v Manchester United Key Moment...,"[[-0.8414877, -0.41296342, -0.35460305, 0.6226..."
5851,Freezing Giant water balloon trial 1 #shorts,Dylan Ayres,215352,5721,https://i.ytimg.com/vi/K-pajCUn35s/default.jpg,#shortsOriginal full video: https://m.youtube....,Freezing Giant water balloon trial 1 shorts,"[[-0.77440363, -0.23070437, 0.19003238, 0.4384..."


In [53]:
import numpy as np

In [54]:
def compute_cosine_similarity(embedding,embeddings):
  similarities=cosine_similarity(embedding.reshape(1,-1),np.vstack(embeddings)).flatten()
  return similarities
def recommended_videos(title,df,tokenizer,model,top_n=8):
  cleaned_title=re.sub(r'[^a-zA-Z0-9]+',' ',title.lower())
  title_embeddings=get_bert_embeddings(cleaned_title,tokenizer,model)
  similarities=compute_cosine_similarity(title_embeddings,df['embeddings']).tolist()
  df['similarity']=similarities
  df_sorted=recommended_videos=df.sort_values('similarity',ascending=False)
  recommendations = df_sorted[df_sorted['title'] != title].head(top_n)
  return recommendations[['title','channelTitle','likes','dislikes','thumbnail_link','description','similarity']]


In [55]:
title_to_search = input("Enter a title to search for: ")
top_recommended_videos=recommended_videos(title_to_search,df_yt,tokenizer,model)

Enter a title to search for: Game Theory: Did Reddit Just SOLVE FNAF?


In [56]:
top_recommended_videos

,title,channelTitle,likes,dislikes,thumbnail_link,description,similarity
2218,Ask Adam Savage: How Lead Balloon Changed Myth...,Adam Savage’s Tested,15438,323,https://i.ytimg.com/vi/h70kbIzPgig/default.jpg,"Tested member Jeffrey McCullough asks Adam, My...",0.990467
7,How Do Nuclear Submarines Make Oxygen?- Smarte...,SmarterEveryDay,57637,554,https://i.ytimg.com/vi/g3Ud6mHdhlQ/default.jpg,Go to https://buyraycon.com/smarter for 15% of...,0.990460
5623,Michael Reeves Guessing Who in Lilypichu Bean ...,OTV Live,10352,83,https://i.ytimg.com/vi/lMz6zKoQRoQ/default.jpg,Subscribe: https://bit.ly/3kmNk4Xhttps://twitc...,0.989779
184,Something About Smash Bros THE SUBSPACE EMISSA...,TerminalMontage,407664,3003,https://i.ytimg.com/vi/5juBmmXoW7I/default.jpg,Kirbo and Luigi's investigation has finally le...,0.987179
4646,Onion Rings - You Suck at Cooking (episode 124),You Suck At Cooking,35717,211,https://i.ytimg.com/vi/rrEe66ZnClQ/default.jpg,Thank you to Bushmills Whiskey for sponsoring ...,0.986986
3005,First Take reacts to Blake Griffin to the Nets,ESPN,17400,480,https://i.ytimg.com/vi/mbwMspaiUVg/default.jpg,First Take reacts to Blake Griffin to the Nets...,0.986414
64,Adorable puppy crashes reporter's weather repo...,FOX 5 Washington DC,7573,249,https://i.ytimg.com/vi/VN32xd-ZAf8/default.jpg,FOX 5's Bob Barnard was cleaning icy cars – an...,0.986008
5413,25 minutes of Rainbow Six Siege VIBING,Marley,58781,448,https://i.ytimg.com/vi/67bBqno5SCM/default.jpg,Thanks to Monster Legends for sponsoring! Don'...,0.985619


In [57]:
title_to_search = input("Enter a title to search for: ")
top_recommended_videos=recommended_videos(title_to_search,df_yt,tokenizer,model)
top_recommended_videos

Enter a title to search for: Action


,title,channelTitle,likes,dislikes,thumbnail_link,description,similarity
5800,Giant Egg Drop | OT 25,Dude Perfect,231552,2864,https://i.ytimg.com/vi/8Mc6zZxBTLA/default.jpg,Egg Drop Challenge featuring GIANT OSTRICH EGG...,0.989981
235,WE LOST OUR PUPPY,Robby and Penny,7981,214,https://i.ytimg.com/vi/wB0nsdQp9Do/default.jpg,So having a new puppy is hard. Much like takin...,0.988016
1600,MY FORTNITE SKIN REVEAL,LazarBeam,369708,2408,https://i.ytimg.com/vi/_PUEb8ppPbg/default.jpg,LazarBeam X Fortnite - CODE LAZARThanks for he...,0.987561
1618,ATEEZ(에이티즈) - ‘불놀이야 (I'm The One)’ Official MV,KQ ENTERTAINMENT,632183,10537,https://i.ytimg.com/vi/xtcVwuauVV0/default.jpg,Melon ▶ https://kko.to/aj3brJeDogenie ▶ https:...,0.987339
5075,Our New Channel!,SML,36947,453,https://i.ytimg.com/vi/L5bIL0YKHR0/default.jpg,This is the new SML channel! Subscribe!Go subs...,0.985917
2229,Dream Gets Robbed...,Dream Shorts,201732,1005,https://i.ytimg.com/vi/RW9tP8HJdx4/default.jpg,Dream Gets Robbed...#shortsSUBSCRIBE! Lots of ...,0.985848
80,Why NOBODY Bought These,Donut Media,48454,803,https://i.ytimg.com/vi/9b_Vt5lja00/default.jpg,Thanks to Dr Squatch for sponsoring today’s ep...,0.985170
208,If Apple Made a Restaurant,Kyle Exum,129623,602,https://i.ytimg.com/vi/6hYLOfy6KP8/default.jpg,Subscribe to my Mini Skits Channel: https://yo...,0.984954


In [58]:
title_to_search = input("Enter a title to search for: ")
top_recommended_videos=recommended_videos(title_to_search,df_yt,tokenizer,model)
top_recommended_videos

Enter a title to search for: Romantic


,title,channelTitle,likes,dislikes,thumbnail_link,description,similarity
1600,MY FORTNITE SKIN REVEAL,LazarBeam,369708,2408,https://i.ytimg.com/vi/_PUEb8ppPbg/default.jpg,LazarBeam X Fortnite - CODE LAZARThanks for he...,0.991439
235,WE LOST OUR PUPPY,Robby and Penny,7981,214,https://i.ytimg.com/vi/wB0nsdQp9Do/default.jpg,So having a new puppy is hard. Much like takin...,0.990730
80,Why NOBODY Bought These,Donut Media,48454,803,https://i.ytimg.com/vi/9b_Vt5lja00/default.jpg,Thanks to Dr Squatch for sponsoring today’s ep...,0.990068
5800,Giant Egg Drop | OT 25,Dude Perfect,231552,2864,https://i.ytimg.com/vi/8Mc6zZxBTLA/default.jpg,Egg Drop Challenge featuring GIANT OSTRICH EGG...,0.989962
5075,Our New Channel!,SML,36947,453,https://i.ytimg.com/vi/L5bIL0YKHR0/default.jpg,This is the new SML channel! Subscribe!Go subs...,0.989886
4651,Lil Mosey - Try Me,LilMoseyVEVO,52915,465,https://i.ytimg.com/vi/XkUciRZCNng/default.jpg,Music video by Lil Mosey performing Try Me. © ...,0.989511
3423,my brother Van,CaseyNeistat,48357,679,https://i.ytimg.com/vi/HgYVuXaihNs/default.jpg,DO IT!!!! -- https://www.youtube.com/user/Va...,0.987911
4428,Diego at the Bank | Awkward Puppets,Awkward Puppets,17339,138,https://i.ytimg.com/vi/tm19P1PMHtA/default.jpg,GET AWKWARD PUPPETS MERCH ► http://lilshop.com...,0.986952


In [59]:
from IPython.display import display, HTML
import pandas as pd

def display_recommendations(recommendations):
    # Generate HTML for displaying thumbnails and metadata
    html = '<div style="display: flex; flex-wrap: wrap; justify-content: space-around;">'

    # Iterate over recommendations in pairs (2 thumbnails per row)
    for i in range(0, len(recommendations), 2):
        row_html = '<div style="display: flex; justify-content: space-around; width: 100%;">'

        # Iterate through each item in the pair (2 thumbnails per row)
        for j in range(2):
            if i + j < len(recommendations):
                # Fetch data for each recommendation
                recommendation = recommendations.iloc[i + j]
                title = recommendation['title']
                thumbnail = recommendation['thumbnail_link']
                likes = recommendation['likes']
                dislikes = recommendation['dislikes']
                similarity = recommendation['similarity']
                channelTitle = recommendation['channelTitle']

                # Generate HTML for each video thumbnail and details
                row_html += f'''
                <div style="width: 20%; margin: 2px; text-align: center; border: 1px solid #ddd; padding: 2px; border-radius: 10px;">
                    <img src="{thumbnail}" alt="{title}" style="width: 100%; border-radius: 5px;">
                    <h4>{title}</h4>
                    <p>Channel: {channelTitle}</p>
                    <p>Likes: {likes} | Dislikes: {dislikes}</p>
                    <p>Similarity: {similarity:.2f}</p>
                </div>
                '''
        row_html += '</div>'
        html += row_html

    html += '</div>'
    display(HTML(html))

In [65]:
title_to_search = input("Enter a title to search for: ")
top_recommended_videos=recommended_videos(title_to_search,df_yt,tokenizer,model)
display_recommendations(top_recommended_videos)

Enter a title to search for: Our New Channel!	
